# Welcome to Python Notebooks and Blender! 👋

This course teaches you all of the tools you need automate repetitive reporting -- starting from the basics: Jupyter, Python and Mito. 

The course focuses on building Python skills that will be helpful to you in practice, leaving behind a lot of the theoretical explanations you might find in other courses. 

The goal of this course is to demistify the Python basics so you have the confidence to start applying Python automations in your day-to-day work.

Things to know before we start:
* 💡 -> the lightbulb emoji means thers an **important information**.
* 🧑‍💻 -> the Person Behind Computer emoji means thers an **task** waiting for you.
* This is not an exam, it's an opportunity to learn something new. You will get stuck at some point -- don't hesitate to ask questions to the Mito team, your colleagues, or Stack Overflow.
* Most imporantly: Enjoy the joureny!

## Objects in Blender


In [17]:
import bpy

💡 The below code will select the default cube and create a variable

In [18]:
cube = bpy.context.active_object
print(cube)

<bpy_struct, Object("Cube") at 0x14474f008>


Now, we display attributes of the cube into the output.   
🧑‍💻 Run `cube.location` , `cube.scale` and `len(cube.data.vertices)`.  
💡 After writing `cube.` followed by the "Tab" key you will get suggestions from auto complete.  


In [19]:
# TODO replace me with your code

🧑‍💻  Now, manipulate the cube.
1. Set the z location with `cube.location.z = 2`
2. Scale the cube with `cube.scale = (1, 1, 2)`
3. 

In [20]:
# TODO replace me with your code


💡 We can also add default objects (torus, monkey, plane, ...) and select them with `obj = bpy.context.active_object`

In [34]:
bpy.ops.mesh.primitive_monkey_add(size=2.5, location=(0, 0, 3))
monkey = bpy.context.active_object

bpy.ops.mesh.primitive_torus_add(major_radius=2, minor_radius=0.3, location=(0,0, 2))
tours = bpy.context.active_object

bpy.ops.mesh.primitive_plane_add(size=3, location=(0, 0, 1.1))  
plane = bpy.context.active_object

💡 Objects can also be removed

In [35]:
bpy.data.objects.remove(tours)

 🧑‍💻 Now, delete the monkey and the plane as well.

In [23]:
# TODO replace me with your code